In [316]:
# analyze who was likely to survive

In [317]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score
from pandas.plotting import scatter_matrix
%matplotlib inline

In [318]:
df = pd.read_csv('/Users/gabrielestes/dsi/python-numpy-pandas-evaluation/data/train.csv')

In [319]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [320]:
df = df.drop(columns=['PassengerId'])

In [321]:
sex = pd.get_dummies(df.Sex)
df = pd.concat([df, sex], axis=1)

In [322]:
location = pd.get_dummies(df.Embarked)
df = pd.concat([df, location], axis=1)

In [323]:
higher_cabins = pd.get_dummies(df.Cabin.str.startswith(('A', 'B', 'C')), 'HigherCabin')

In [324]:
df = pd.concat([df, higher_cabins], axis=1)
df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,female,male,C,Q,S,HigherCabin_False,HigherCabin_True
0,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0,1,0,0,1,0,0
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1,0,1,0,0,0,1
2,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1,0,0,0,1,0,0
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1,0,0,0,1,0,1
4,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0,1,0,0,1,0,0


In [325]:
df.corr()

,Survived,Pclass,Age,SibSp,Parch,Fare,female,male,C,Q,S,HigherCabin_False,HigherCabin_True
Survived,1.000000,-0.338481,-0.077221,-0.035322,0.081629,0.257307,0.543351,-0.543351,0.168240,0.003650,-0.155660,0.215509,0.205840
Pclass,-0.338481,1.000000,-0.369226,0.083081,0.018443,-0.549500,-0.131900,0.131900,-0.243292,0.221009,0.081720,-0.317095,-0.620824
Age,-0.077221,-0.369226,1.000000,-0.308247,-0.189119,0.096067,-0.093254,0.093254,0.036261,-0.022405,-0.032523,0.121361,0.200317
SibSp,-0.035322,0.083081,-0.308247,1.000000,0.414838,0.159651,0.114631,-0.114631,-0.059528,-0.026354,0.070941,-0.036472,-0.018679
Parch,0.081629,0.018443,-0.189119,0.414838,1.000000,0.216225,0.245489,-0.245489,-0.011069,-0.081228,0.063036,0.001571,0.044031
Fare,0.257307,-0.549500,0.096067,0.159651,0.216225,1.000000,0.182333,-0.182333,0.269335,-0.117216,-0.166603,0.079450,0.523843
female,0.543351,-0.131900,-0.093254,0.114631,0.245489,0.182333,1.000000,-1.000000,0.082853,0.074115,-0.125722,0.103058,0.084749
male,-0.543351,0.131900,0.093254,-0.114631,-0.245489,-0.182333,-1.000000,1.000000,-0.082853,-0.074115,0.125722,-0.103058,-0.084749
C,0.168240,-0.243292,0.036261,-0.059528,-0.011069,0.269335,0.082853,-0.082853,1.000000,-0.148258,-0.778359,0.033073,0.227692
Q,0.003650,0.221009,-0.022405,-0.026354,-0.081228,-0.117216,0.074115,-0.074115,-0.148258,1.000000,-0.496624,-0.071089,-0.098602


In [326]:
# female, male, Fare, and Pclass are contenders

In [327]:
y = df.Survived
y.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

In [328]:
X = df[['Pclass', 'Age', 'SibSp', 'Survived', 'female', 'S', 'Q', 'C', 'HigherCabin_False', 'HigherCabin_True']].copy()
X.Age = X.Age.fillna(X.Age.mean())
X = X.drop(columns=['Survived'])

X.head()

,Pclass,Age,SibSp,female,S,Q,C,HigherCabin_False,HigherCabin_True
0,3,22.0,1,0,1,0,0,0,0
1,1,38.0,1,1,0,0,1,0,1
2,3,26.0,0,1,1,0,0,0,0
3,1,35.0,1,1,1,0,0,0,1
4,3,35.0,0,0,1,0,0,0,0


In [329]:
model = sm.OLS(y, X)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               Survived   R-squared:                       0.630
Model:                            OLS   Adj. R-squared:                  0.626
Method:                 Least Squares   F-statistic:                     166.6
Date:                Sun, 22 Jul 2018   Prob (F-statistic):          1.93e-183
Time:                        18:54:19   Log-Likelihood:                -395.25
No. Observations:                 891   AIC:                             808.5
Df Residuals:                     882   BIC:                             851.6
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Pclass               -0.1300      0.023     -5.629      0.000      -0.175      -0.085
Age                  -0.0052      0.001     -4.930      0.000      -0.007      -0.003
SibSp                -0.0421      0.012     -3.522      0.000      -0.066      -0.019
female                0.5041      0.027     18.409      0.000       0.450       0.558
S                     0.6322      0.076      8.346      0.000       0.484       0.781
Q                     0.6932      0.093      7.415      0.000       0.510       0.877
C                     0.7057      0.078      9.092      0.000       0.553       0.858
HigherCabin_False     0.1922      0.051      3.769      0.000       0.092       0.292
HigherCabin_True      0.0890      0.052      1.726      0.085      -0.012       0.190
==============================================================================
Omnibus:                       37.019   Durbin-Watson:                   1.925
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               40.614
Skew:                           0.517   Prob(JB):                     1.52e-09
Kurtosis:                       3.158   Cond. No.                         365.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [330]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [331]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [332]:
model.predict(X_test)

array([0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 1, 1, 0])

In [333]:
model.score(X_test, y_test)

0.8135593220338984

In [334]:
results.params

Pclass              -0.130034
Age                 -0.005161
SibSp               -0.042114
female               0.504132
S                    0.632194
Q                    0.693188
C                    0.705677
HigherCabin_False    0.192233
HigherCabin_True     0.088969
dtype: float64

In [335]:
y_hat = results.predict(X_test)
r2_score(y_test, y_hat)

0.4497974684864058